# Import Relevant Libraries

In [3]:
import requests
import bs4 as bs
import pandas as pd
import os
from fake_useragent import UserAgent

# Environment Specifics

In [4]:
print('python version: ', os.sys.version)
print('pandas version: ', pd.__version__)
print('requests version: ', requests.__version__)
print('bs4 version: ', bs.__version__)

python version:  3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:38:29) [Clang 13.0.1 ]
pandas version:  1.5.0
requests version:  2.28.1
bs4 version:  4.11.1


# Specify Relevant URLs

The cell below contains the SEC and Wikipedia URLs used to scrape the data. A detailed description of the process followed can be found in the accompanying report.

In [2]:
# Relevant URLs

# URL for the SEC's list of CIKs and tickers
cik_mapping_url = 'https://www.sec.gov/files/company_tickers.json'
# URL for the SEC's list of filings for a given CIK
doc_10K_index_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=10-K&dateb=&owner=exclude&count=10'
doc_10KA_index_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=10-K%2FA&dateb=&owner=exclude&count=10'
# SP500 data
sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Helper Functions

Helper functions are defined below. These functions are used to scrape the data from the SEC and Wikipedia. Each function is carefully documented.

In [3]:
# Helper functions
def index_filter(tag: bs.element.Tag) -> str:
    """Filter tags to identify the 10-K document links

    Args:
        tag (bs.element.Tag): Tag to be inspected

    Returns:
        str: True if the tag is a link to a 10-K document, None otherwise
    """
    if tag.name == 'a' and tag.has_attr('href'):
        href = tag['href']
        return href.startswith('/Archives/edgar/data/') and href.endswith('-index.htm')
    return False

def get_10K_url(cik: str, headers: dict) -> str:
    """Given a CIK, get the URL of the most recent 10-K document.

    Args:
        cik (str): CIK identifier
        headers (dict): Headers to be used in the request

    Returns:
        str: URL of the most recent 10-K document, None if no 10-K document is found
    """
    # Get 10-Ks
    response_10K = requests.get(url=doc_10K_index_url.format(cik=cik), headers=headers)
    soup_10K = bs.BeautifulSoup(response_10K.text, 'lxml')
    tags_10K = soup_10K.find_all(index_filter)
    # Get 10-K/As (amendments)
    response_10KA = requests.get(url=doc_10KA_index_url.format(cik=cik), headers=headers)
    soup_10KA = bs.BeautifulSoup(response_10KA.text, 'lxml')
    tags_10KA = soup_10KA.find_all(index_filter)
    # Remove 10-K/As from 10-Ks
    tags = [tag for tag in tags_10K if tag not in tags_10KA]
    # Return URL of most recent 10-K, if any
    if len(tags) > 0:
        return 'https://www.sec.gov' + tags[0]['href'][:-10] + '.txt'
    return None

# Scrape SEC Data

The cell below scrapes the SEC data. The data is scraped from the SEC website and saved as a CSV file. A detailed description of the process followed can be found in the accompanying report.

In [4]:
# Set headers
headers = {'User-Agent': UserAgent().chrome}

# Get the list of tickers from the SEC website, together with the company names and CIKs
response = requests.get(url=cik_mapping_url, headers=headers)
ciks = pd.DataFrame(response.json()).transpose()
ciks = ciks.rename(columns={'cik_str': 'cik', 'title': 'name'})

# Get SP500 data
sp500 = pd.read_html(sp500_url)[0][['Symbol', 'Security', 'Date added', 'GICS Sector', 'GICS Sub-Industry']].copy()
sp500.columns = ['ticker', 'name', 'added', 'sector', 'sub_industry']

# Only consider SP500 companies
mappings = ciks.merge(sp500, on='ticker', suffixes=('_sec', '_sp500'))
# Some companies have multiple tickers (Class A vs Class B shares) which however share the same CIK
mappings = mappings.drop_duplicates(subset=['cik'])

# For every CIK, get the URL of the most recent 10-K document
doc_urls = {'cik': [], 'url' : []}
for cik in mappings['cik'].to_list():
    doc_urls['cik'].append(cik)
    doc_urls['url'].append(get_10K_url(cik, headers))
doc_urls = pd.DataFrame(doc_urls)

# Add the URLs to the mappings
mappings = mappings.merge(doc_urls, on='cik')

# Save the mappings
mappings.to_csv('mappings.csv', index=False)

In [5]:
# Show mappings
mappings.head()

,cik,ticker,name_sec,name_sp500,added,sector,sub_industry,url
0,320193,AAPL,Apple Inc.,Apple Inc.,1982-11-30,Information Technology,"Technology Hardware, Storage & Peripherals",https://www.sec.gov/Archives/edgar/data/320193...
1,789019,MSFT,MICROSOFT CORP,Microsoft,1994-06-01,Information Technology,Systems Software,https://www.sec.gov/Archives/edgar/data/789019...
2,1652044,GOOGL,Alphabet Inc.,Alphabet Inc. (Class A),2014-04-03,Communication Services,Interactive Media & Services,https://www.sec.gov/Archives/edgar/data/165204...
3,1018724,AMZN,AMAZON COM INC,Amazon,2005-11-18,Consumer Discretionary,Broadline Retail,https://www.sec.gov/Archives/edgar/data/101872...
4,1045810,NVDA,NVIDIA CORP,Nvidia,2001-11-30,Information Technology,Semiconductors,https://www.sec.gov/Archives/edgar/data/104581...
